In [1]:
import pandas as pd

output_correct = pd.DataFrame()

output_error = pd.DataFrame()

In [2]:
input_ = pd.read_excel('./part_number.xlsx',
                       header=0,
                       dtype=str).fillna('')

input_ = input_[['JOIN_MPN', 'Part_Type', 'Part_Key']]
input_.drop_duplicates(['JOIN_MPN'],
                       keep='first',
                       ignore_index=True)

length = len(input_)

print('总数量：' + str(length))
input_['JOIN_MPN'].tolist()

总数量：475


['MELLING;30250S',
 'MELLING;30250SX',
 'MELLING;30420S',
 'MELLING;30430S',
 'MELLING;30460S',
 'MELLING;30460SA',
 'MELLING;30460SB',
 'MELLING;30470S',
 'MELLING;30470SX',
 'MELLING;30490S',
 'MELLING;30490SA',
 'MELLING;30490SAX',
 'MELLING;30490SX',
 'MELLING;30719S',
 'MELLING;30719SHWP',
 'MELLING;31007S',
 'MELLING;31035S',
 'MELLING;31035SX',
 'MELLING;31036S',
 'MELLING;31036SA',
 'MELLING;31038S',
 'MELLING;31038SX',
 'MELLING;31041S',
 'MELLING;31045S',
 'MELLING;31045SA',
 'MELLING;31047S',
 'MELLING;31047SH',
 'MELLING;31047SHWP',
 'MELLING;31047SX',
 'MELLING;31047SXH',
 'MELLING;31049S',
 'MELLING;31049SA',
 'MELLING;31049SAH',
 'MELLING;31049SAHWP',
 'MELLING;31049SAW',
 'MELLING;31049SB',
 'MELLING;31049SBH',
 'MELLING;31049SBX',
 'MELLING;31049SBXH',
 'MELLING;31049SC',
 'MELLING;31049SCH',
 'MELLING;31049SCX',
 'MELLING;31049SCXH',
 'MELLING;31049SD',
 'MELLING;31049SDH',
 'MELLING;31049SDX',
 'MELLING;31049SDXH',
 'MELLING;31049SE',
 'MELLING;31049SEH',
 'MELLING;3

In [3]:
crawler_status = 'error'

print(crawler_status)

error


In [4]:
import requests
import json

In [5]:
import sys
sys.path.append('../../00.module')
import UA
import Proxy

i = 1

jsn = '{"partData":{"listing_data_essential":{"parttype":"' + input_.loc[i, 'Part_Type'] + '","partkey":"' + input_.loc[i, 'Part_Key'] + '"}}}'

payload = {'func': 'getbuyersguide',
           'payload': jsn,
           'api_json_request': '1'}

resp = requests.post('https://www.rockauto.com/catalog/catalogapi.php',
                     data=payload,
                     headers=UA.get_User_Agent_Requests(),
                     proxies=Proxy.get_Proxy_Requests()).text

resp = json.loads(resp)

resp

{'buyersguidepieces': {'body': '<div class="buyersguide-nested"><div style="padding: .5em;"><table class="nobmp" style="width: 100%;"><tr><td class="altrow-a-0">MAZDA</td><td class="altrow-a-0">3</td><td class="altrow-a-0">2004-2007</td></tr><tr><td class="altrow-a-1">MAZDA</td><td class="altrow-a-1">5</td><td class="altrow-a-1">2006-2007</td></tr><tr><td class="altrow-a-0">MAZDA</td><td class="altrow-a-0">6</td><td class="altrow-a-0">2003-2007</td></tr></table><div>Please refer to catalog for application details.</div></div></div>',
  'title': "Buyer's Guide :   Timing Chain & Component Kit"},
 'groupindex': None,
 'collected_javascript': '/**UQ:E2780280**/\nwindow.top.parent.window["_nck"] = "8LVLUxTaILBT+tLxkg8+3Ag7\\/0bbOhvn7P3rkFbL80\\/ismhHyfCiO07pwNUCaIW3afbJMKR0uqi1pNnUlRgBN5CwkZnWFYwm7DtiVw8cl6t1DDIlw0OPK8L1xqUF2GhWh7smW3GvoaMDTrbM9hXPpFBLxYmBSG1URmNkfXxHdpk=";\n'}

In [6]:
from bs4 import BeautifulSoup

In [7]:
html = resp['buyersguidepieces']['body']

soup = BeautifulSoup(html, 'lxml')

with open('./vehicle.html', 'w', encoding='UTF-8') as file:
    file.write(str(soup))

print(soup.prettify())

<html>
 <body>
  <div class="buyersguide-nested">
   <div style="padding: .5em;">
    <table class="nobmp" style="width: 100%;">
     <tr>
      <td class="altrow-a-0">
       MAZDA
      </td>
      <td class="altrow-a-0">
       3
      </td>
      <td class="altrow-a-0">
       2004-2007
      </td>
     </tr>
     <tr>
      <td class="altrow-a-1">
       MAZDA
      </td>
      <td class="altrow-a-1">
       5
      </td>
      <td class="altrow-a-1">
       2006-2007
      </td>
     </tr>
     <tr>
      <td class="altrow-a-0">
       MAZDA
      </td>
      <td class="altrow-a-0">
       6
      </td>
      <td class="altrow-a-0">
       2003-2007
      </td>
     </tr>
    </table>
    <div>
     Please refer to catalog for application details.
    </div>
   </div>
  </div>
 </body>
</html>



In [8]:
from lxml import etree

In [9]:
html = etree.HTML(str(soup))

security_code = html.xpath('//div[@class="center"]/div')

security_code

[]

In [10]:
len(security_code) != 0 and 'security code' in security_code[0].text

False

In [11]:
list_vehicle = []
list_tr = html.xpath('//div[contains(@class, "buyersguide-nested")]/div/table[contains(@class, "nobmp")]/tr')
for tr in list_tr:
    list_td = [text.strip() for text in tr.xpath('./td/text()')]
    list_vehicle.append(' '.join(list_td).strip())

vehicle = '\n'.join(list_vehicle)

print(vehicle)

MAZDA 3 2004-2007
MAZDA 5 2006-2007
MAZDA 6 2003-2007


In [12]:
df_temp = pd.DataFrame([{'JOIN_MPN': input_.loc[i, 'JOIN_MPN'],
                         'Vehicle': vehicle}])

output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

output_correct

,JOIN_MPN,Vehicle
0,MELLING;30250SX,MAZDA 3 2004-2007\nMAZDA 5 2006-2007\nMAZDA 6 ...


In [13]:
crawler_status = 'ok'

print(crawler_status)

ok


In [14]:
if crawler_status != 'ok':
    df_temp = pd.DataFrame([{'JOIN_MPN': input_.loc[i, 'JOIN_MPN'],
                             'Part_Type': input_.loc[i, 'Part_Type'],
                             'Part_Key': input_.loc[i, 'Part_Key']}])
    
    output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')
    
output_error

""
